In [2]:
import pandas as pd
import numpy as np
import sys
import os
import bs4
from IPython.display import display_html,clear_output, HTML
import re
from datetime import datetime
import ast
import itertools
from tqdm import tqdm,trange
from file_tools import *
from request_tools import *
from parse_tools import *
from load_tools import *
from model_tools import *

pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 30)
pd.set_option('compute.use_numexpr', False)

import warnings
from pandas.errors import PerformanceWarning
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning
pd.options.mode.use_inf_as_na = True
warnings.filterwarnings('ignore', category=PerformanceWarning)
warnings.filterwarnings('ignore', category=DataConversionWarning)
warnings.filterwarnings('ignore', category=ConvergenceWarning)

import pandas as pd
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif, RFE, mutual_info_regression
from sklearn.linear_model import LogisticRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from scipy.stats import pearsonr
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVR

In [3]:
DATA_DICT = load_all_df_from_dir('./07-data-compiled/03-boxscores-players-stats-1st-vs-2nd-half-1997-2023/',index_col=[0,1,2],header=[0,1])
DATA_DICT.keys()

dict_keys(['boxscores_players_1st_half', 'boxscores_players_2nd_half'])

In [4]:
boxscores_players_1st_half = DATA_DICT['boxscores_players_1st_half']
boxscores_players_2nd_half = DATA_DICT['boxscores_players_2nd_half']
boxscores_players_1st_half

H1   
                                                                                    mp   
boxscore_id                  team_id              player_id                              
/boxscores/202304090WAS.html /teams/HOU/2023.html /players/s/smithja05.html  15.033333  \
                                                  /players/c/chrisjo01.html  12.183333   
                                                  /players/g/greenja05.html  15.150000   
                                                  /players/s/sengual01.html  13.366667   
                                                  /players/m/martike04.html  10.866667   
                                                  /players/e/easonta01.html  13.133333   
                                                  /players/w/washity02.html   8.850000   
                                                  /players/n/nixda01.html    11.816667   
                                                  /players/k/kaminfr01.html   8.966667   
                                                  /players/m/marjabo01.html   4.733333   
                                                  /players/g/garubus01.html   5.900000   
                                                  /players/a/augusdj01.html   0.000000   
                                                  /players/c/caulewi01.html   0.000000   
                                                  /players/d/daysda01.html    0.000000   
                                                  /players/h/hudgitr01.html   0.000000   
...                                                                                ...   
/boxscores/199611010BOS.html /teams/CHI/1997.html /players/k/kerrst01.html    8.500000   
                                                  /players/w/wennibi01.html   5.616667   
                                                  /players/p/parisro01.html   0.000000   
                                                  /players/b/brownra02.html   2.650000   
                                                  /players/s/simpkdi01.html   4.700000   
                                                  /players/b/buechju01.html   0.600000   
                             /teams/BOS/1997.html /players/w/wesleda01.html  18.483333   
                                                  /players/f/foxri01.html    16.566667   
                                                  /players/e/ellispe01.html  15.050000   
                                                  /players/w/walkean02.html  18.250000   
                                                  /players/w/willier01.html  12.700000   
                                                  /players/b/barroda01.html  16.316667   
                                                  /players/d/dayto01.html     9.466667   
                                                  /players/b/brickfr01.html  11.300000   
                                                  /players/s/szabobr01.html   1.866667   

                                                                                   
                                                                              fg   
boxscore_id                  team_id              player_id                        
/boxscores/202304090WAS.html /teams/HOU/2023.html /players/s/smithja05.html  4.0  \
                                                  /players/c/chrisjo01.html  3.0   
                                                  /players/g/greenja05.html  2.0   
                                                  /players/s/sengual01.html  0.0   
                                                  /players/m/martike04.html  1.0   
                                                  /players/e/easonta01.html  3.0   
                                                  /players/w/washity02.html  3.0   
                                                  /players/n/nixda01.html    1.0   
                                                  /players/k/kaminfr01.html  1.0   
               

In [5]:
boxscores_players_2nd_half

GM   
                                                                                    mp   
boxscore_id                  team_id              player_id                              
/boxscores/202304090WAS.html /teams/HOU/2023.html /players/s/smithja05.html  31.016667  \
                                                  /players/c/chrisjo01.html  30.250000   
                                                  /players/g/greenja05.html  27.150000   
                                                  /players/s/sengual01.html  23.533333   
                                                  /players/m/martike04.html  21.033333   
                                                  /players/e/easonta01.html  26.966667   
                                                  /players/w/washity02.html  20.850000   
                                                  /players/n/nixda01.html    17.750000   
                                                  /players/k/kaminfr01.html  16.983333   
                                                  /players/m/marjabo01.html  12.633333   
                                                  /players/g/garubus01.html  11.833333   
                                                  /players/a/augusdj01.html   0.000000   
                                                  /players/c/caulewi01.html   0.000000   
                                                  /players/d/daysda01.html    0.000000   
                                                  /players/h/hudgitr01.html   0.000000   
...                                                                                ...   
/boxscores/199611010BOS.html /teams/CHI/1997.html /players/k/kerrst01.html   20.983333   
                                                  /players/w/wennibi01.html  10.133333   
                                                  /players/p/parisro01.html   8.716667   
                                                  /players/b/brownra02.html   5.383333   
                                                  /players/s/simpkdi01.html   4.700000   
                                                  /players/b/buechju01.html   0.616667   
                             /teams/BOS/1997.html /players/w/wesleda01.html  40.783333   
                                                  /players/f/foxri01.html    37.383333   
                                                  /players/e/ellispe01.html  31.133333   
                                                  /players/w/walkean02.html  26.583333   
                                                  /players/w/willier01.html  25.400000   
                                                  /players/b/barroda01.html  37.716667   
                                                  /players/d/dayto01.html    20.250000   
                                                  /players/b/brickfr01.html  18.883333   
                                                  /players/s/szabobr01.html   1.866667   

                                                                                   
                                                                              fg   
boxscore_id                  team_id              player_id                        
/boxscores/202304090WAS.html /teams/HOU/2023.html /players/s/smithja05.html  6.0  \
                                                  /players/c/chrisjo01.html  6.0   
                                                  /players/g/greenja05.html  5.0   
                                                  /players/s/sengual01.html  4.0   
                                                  /players/m/martike04.html  5.0   
                                                  /players/e/easonta01.html  6.0   
                                                  /players/w/washity02.html  3.0   
                                                  /players/n/nixda01.html    1.0   
                                                  /players/k/kaminfr01.html  3.0   
               

In [6]:
# Seasomn Filter
MP_FILTER = (boxscores_players_1st_half[('H1','mp')] >= 5) & (boxscores_players_2nd_half[('GM','mp')] >= 20)
X_BXSC_PLYR = boxscores_players_1st_half.loc[MP_FILTER].drop(columns=['Opp_H1'])
Y_BXSC_PLYR = boxscores_players_2nd_half.loc[MP_FILTER,('GM','pts')]

YEAR_FILTER = Y_BXSC_PLYR.index.to_frame()['team_id'].str[-9:-5].astype(int)
X = X_BXSC_PLYR.loc[YEAR_FILTER<2010].iloc[:10000,:]
Y= Y_BXSC_PLYR.loc[YEAR_FILTER<2010].iloc[:10000]

X

H1   
                                                                                    mp   
boxscore_id                  team_id              player_id                              
/boxscores/200904150SAS.html /teams/NOH/2009.html /players/w/westda01.html   19.333333  \
                                                  /players/p/paulch01.html   18.316667   
                                                  /players/s/stojape01.html  19.000000   
                                                  /players/b/butlera01.html  19.033333   
                                                  /players/c/chandty01.html  20.183333   
                                                  /players/p/poseyja01.html   9.633333   
                             /teams/SAS/2009.html /players/p/parketo01.html  17.916667   
                                                  /players/f/finlemi01.html  14.916667   
                                                  /players/m/masonro01.html  16.116667   
                                                  /players/d/duncati01.html  12.666667   
                                                  /players/b/bonnema01.html  14.700000   
                                                  /players/u/udokaim01.html  18.016667   
/boxscores/200904150POR.html /teams/DEN/2009.html /players/b/billuch01.html  18.400000   
                                                  /players/a/anthoca01.html  17.616667   
                                                  /players/h/hilarne01.html  18.033333   
...                                                                                ...   
/boxscores/200812260MIA.html /teams/MIA/2009.html /players/c/chalmma01.html  17.333333   
                                                  /players/w/wadedw01.html   16.916667   
                                                  /players/h/hasleud01.html  14.750000   
                                                  /players/m/mariosh01.html  16.983333   
                                                  /players/a/anthojo01.html  18.650000   
                                                  /players/c/cookda02.html   19.083333   
                                                  /players/b/beaslmi01.html   9.616667   
/boxscores/200812260MEM.html /teams/IND/2009.html /players/d/daniema01.html  17.650000   
                                                  /players/m/murphtr01.html  15.500000   
                                                  /players/j/jackja01.html   16.600000   
                                                  /players/g/grangda01.html  17.833333   
                                                  /players/r/rushbr01.html   14.516667   
                                                  /players/n/nestera01.html  14.116667   
                             /teams/MEM/2009.html /players/m/mayooj01.html   20.016667   
                                                  /players/g/gayru01.html    19.933333   

                                                                                   
                                                                              fg   
boxscore_id                  team_id              player_id                        
/boxscores/200904150SAS.html /teams/NOH/2009.html /players/w/westda01.html   8.0  \
                                                  /players/p/paulch01.html   4.0   
                                                  /players/s/stojape01.html  0.0   
                                                  /players/b/butlera01.html  3.0   
                                                  /players/c/chandty01.html  4.0   
                                                  /players/p/poseyja01.html  1.0   
                             /teams/SAS/2009.html /players/p/parketo01.html  7.0   
                                                  /players/f/finlemi01.html  3.0   
                                                  /players/m/masonro01.html  2.0   
               

Try predict by player

In [12]:
# grouped = X_BXSC_PLYR.iloc[:10000].groupby('player_id')
# df_dict = {name: group for name, group in grouped}

model_stack = ModelStackCV(models=ModelStackCV.MODELS_REGRESSION,model_params=ModelStackCV.MODEL_PARAMS_REGRESSION)
predictions,scores = model_stack.run_experiment(
    X={key: data for key, data in X_BXSC_PLYR.loc[(slice(None),slice(None),['/players/g/gayru01.html','/players/p/paulch01.html','/players/c/chandty01.html']),:].groupby('player_id')},
    y={key: data for key, data in Y_BXSC_PLYR.loc[(slice(None),slice(None),['/players/g/gayru01.html','/players/p/paulch01.html','/players/c/chandty01.html'])].groupby('player_id')},
    cv=3, scaler=StandardScaler(),score_names=ModelStackCV.SCORES_REGRESSION,
    save_dir='./test.csv'
)

# df_dict['/players/a/achiupr01.html']

In [10]:
X_BXSC_PLYR.loc[(slice(None),slice(None),['/players/g/gayru01.html','/players/p/paulch01.html','/players/c/chandty01.html']),:]

H1   
                                                                                    mp   
boxscore_id                  team_id              player_id                              
/boxscores/202212200DET.html /teams/UTA/2023.html /players/g/gayru01.html     8.450000  \
/boxscores/202210190UTA.html /teams/UTA/2023.html /players/g/gayru01.html    13.416667   
/boxscores/202204060UTA.html /teams/UTA/2022.html /players/g/gayru01.html    13.066667   
/boxscores/202203270DAL.html /teams/UTA/2022.html /players/g/gayru01.html    11.100000   
/boxscores/202203200NYK.html /teams/UTA/2022.html /players/g/gayru01.html    12.133333   
/boxscores/202203180UTA.html /teams/UTA/2022.html /players/g/gayru01.html    14.516667   
/boxscores/202203140UTA.html /teams/UTA/2022.html /players/g/gayru01.html    12.683333   
/boxscores/202202020UTA.html /teams/UTA/2022.html /players/g/gayru01.html    13.316667   
/boxscores/202201240PHO.html /teams/UTA/2022.html /players/g/gayru01.html    16.633333   
/boxscores/202201160DEN.html /teams/UTA/2022.html /players/g/gayru01.html     9.816667   
/boxscores/202201050DEN.html /teams/UTA/2022.html /players/g/gayru01.html    14.783333   
/boxscores/202201030NOP.html /teams/UTA/2022.html /players/g/gayru01.html    11.750000   
/boxscores/202112310UTA.html /teams/UTA/2022.html /players/g/gayru01.html    12.833333   
/boxscores/202112290POR.html /teams/UTA/2022.html /players/g/gayru01.html    14.516667   
/boxscores/202112180UTA.html /teams/UTA/2022.html /players/g/gayru01.html    10.783333   
...                                                                                ...   
/boxscores/200202260CHI.html /teams/CHI/2002.html /players/c/chandty01.html  14.133333   
/boxscores/200202230CHI.html /teams/CHI/2002.html /players/c/chandty01.html  10.516667   
/boxscores/200202220CHI.html /teams/CHI/2002.html /players/c/chandty01.html  12.800000   
/boxscores/200202200CHI.html /teams/CHI/2002.html /players/c/chandty01.html  20.300000   
/boxscores/200202180MIA.html /teams/CHI/2002.html /players/c/chandty01.html  14.550000   
/boxscores/200202150CHI.html /teams/CHI/2002.html /players/c/chandty01.html  14.083333   
/boxscores/200202120CHI.html /teams/CHI/2002.html /players/c/chandty01.html  12.600000   
/boxscores/200202060LAL.html /teams/CHI/2002.html /players/c/chandty01.html  13.116667   
/boxscores/200202030POR.html /teams/CHI/2002.html /players/c/chandty01.html  14.133333   
/boxscores/200202020SEA.html /teams/CHI/2002.html /players/c/chandty01.html  12.750000   
/boxscores/200201300UTA.html /teams/CHI/2002.html /players/c/chandty01.html   8.366667   
/boxscores/200201150CHI.html /teams/CHI/2002.html /players/c/chandty01.html   9.266667   
/boxscores/200112270DAL.html /teams/CHI/2002.html /players/c/chandty01.html   8.533333   
/boxscores/200112220CHI.html /teams/CHI/2002.html /players/c/chandty01.html  13.083333   
/boxscores/200111070CHI.html /teams/CHI/2002.html /players/c/chandty01.html   6.550000   

                                                                                   
                                                                              fg   
boxscore_id                  team_id              player_id                        
/boxscores/202212200DET.html /teams/UTA/2023.html /players/g/gayru01.html    0.0  \
/boxscores/202210190UTA.html /teams/UTA/2023.html /players/g/gayru01.html    4.0   
/boxscores/202204060UTA.html /teams/UTA/2022.html /players/g/gayru01.html    3.0   
/boxscores/202203270DAL.html /teams/UTA/2022.html /players/g/gayru01.html    4.0   
/boxscores/202203200NYK.html /teams/UTA/2022.html /players/g/gayru01.html    2.0   
/boxscores/202203180UTA.html /teams/UTA/2022.html /players/g/gayru01.html    3.0   
/boxscores/202203140UTA.html /teams/UTA/2022.html /players/g/gayru01.html    1.0   
/boxscores/202202020UTA.html /teams/UTA/2022.html /players/g/gayru01.html    2.0   
/boxscores/202201240PHO.html /teams/UTA/2022.html /players/g/gayru01.html    3.0   
/boxscores/2022

In [13]:
Y_BXSC_PLYR.shape[0]

403223

In [24]:
Y_BXSC_PLYR.groupby('player_id').count()

player_id
/players/a/abdulma02.html     80
/players/a/abdulta01.html    127
/players/a/abdursh01.html    773
/players/a/abrinal01.html     52
/players/a/achiupr01.html     79
/players/a/acyqu01.html      110
/players/a/adamsha01.html      9
/players/a/adamsja01.html      4
/players/a/adamsjo01.html      4
/players/a/adamsst01.html    588
/players/a/addisra01.html      1
/players/a/adebaba01.html    348
/players/a/adelde01.html       3
/players/a/adrieje01.html     38
/players/a/afflaar01.html    569
                            ... 
/players/y/youngja01.html      5
/players/y/youngjo01.html     11
/players/y/youngni01.html    438
/players/y/youngsa01.html     78
/players/y/youngth01.html    974
/players/y/youngtr01.html    348
/players/y/yurtsom01.html     15
/players/z/zelleco01.html    301
/players/z/zellety01.html    168
/players/z/zhizhwa01.html      7
/players/z/zidekge01.html      3
/players/z/zimmede01.html      1
/players/z/zipsepa01.html     34
/players/z/zizican01.html     30


In [22]:
# Group by players with at least 50 appearances in the series
MIN_CAPS = 50
player_appearances = Y_BXSC_PLYR.groupby('player_id').count()
player_appearances = player_appearances[player_appearances >= MIN_CAPS]
MIN_CAPS_FILTER = Y_BXSC_PLYR.index.get_level_values('player_id').isin(player_appearances.index)
# Y_BXSC_PLYR

boxscore_id                   team_id               player_id                
/boxscores/202304090WAS.html  /teams/HOU/2023.html  /players/s/smithja05.html    20.0
                                                    /players/g/greenja05.html    14.0
                                                    /players/s/sengual01.html    12.0
                                                    /players/m/martike04.html    12.0
                              /teams/WAS/2023.html  /players/n/nunnke01.html     12.0
/boxscores/202304090TOR.html  /teams/MIL/2023.html  /players/l/leoname01.html    10.0
                                                    /players/c/crowdja01.html    13.0
                                                    /players/m/matthwe02.html     3.0
                                                    /players/d/dragigo01.html    14.0
                              /teams/TOR/2023.html  /players/a/achiupr01.html    14.0
                                                    /players/t

In [ ]:
# scores_dict = {}
# predictions_dict = {}
# for player_id, df in X_BXSC_PLYR.groupby('player_id'):
#     X = df
#     y = Y_BXSC_PLYR.loc[df.index]
#     model_stack = ModelStackCV(models=ModelStackCV.MODELS_REGRESSION,model_params=ModelStackCV.MODEL_PARAMS_REGRESSION)
#     predictions,scores = model_stack.run_experiment(
#         X=X, y=y, cv=3, scaler=StandardScaler(),score_names=ModelStackCV.SCORES_REGRESSION,save_dir=None
#     )
#     scores_dict[player_id] = scores
#     predictions_dict[player_id] = predictions

#     scores_all = pd.concat(scores_dict,axis=0)
#     predictions_all = pd.concat(predictions_dict,axis=0)
#     scores_all.to_csv('./08-experiments/03-boxscores-players-stats-1st-vs-2nd-half-1997-2023/predict-players-pts-total/scores.csv')
#     predictions_all.to_csv('./08-experiments/03-boxscores-players-stats-1st-vs-2nd-half-1997-2023/predict-players-pts-total/predictions.csv')

# scores_all.to_csv('./08-experiments/03-boxscores-players-stats-1st-vs-2nd-half-1997-2023/predict-players-pts-total/scores.csv')
# predictions_all.to_csv('./08-experiments/03-boxscores-players-stats-1st-vs-2nd-half-1997-2023/predict-players-pts-total/predictions.csv')
# scores

    

In [ ]:
# def run_experiment(
#     model_stack,
#     X,
#     y,
#     cv,
#     scaler,
#     score_names,
#     save_dir = None,
#     append_to = None, # Append to existing predictions and scores before saving
#     experiment_name='experiment',
# ):

#     # Run Experiment
#     if isinstance(X,dict) and isinstance(y,dict):
#         predictions = {}
#         scores = {}
#         for name,df in X.items():
#             predictions[name] = model_stack.fit_predict(df,y,cv=cv,scaler=scaler)
#             scores[name] = model_stack.score(predictions[name], score_names)
#         predictions = pd.concat(predictions,axis=0)
#         scores = pd.concat(scores,axis=0)
#     else:
#         predictions = model_stack.fit_predict(X,y,cv=cv,scaler=scaler)
#         scores = model_stack.score(predictions, score_names)

#     # Save Predictions
#     if save_dir:
#         experiment_name = datetime.now().strftime('%Y%m%d-%H%M%S')+f'-{experiment_name}'
#         save_dir = os.path.join(save_dir,experiment_name)
#         make_directory(save_dir)
#         predictions.to_csv(os.path.join(save_dir,'predictions.csv'))
#         scores.to_csv(os.path.join(save_dir,'scores.csv'))
#     return predictions, scores 

# for player_id, df in X_BXSC_PLYR.groupby('player_id'):
#     X = df
#     y = Y_BXSC_PLYR.loc[df.index]

#     model_stack = ModelStackCV(models=ModelStackCV.MODELS_REGRESSION,model_params=ModelStackCV.MODEL_PARAMS_REGRESSION)
#     predictions,scores = run_experiment(
#         model_stack=model_stack,
#         X=X, y=Y, cv=3, scaler=StandardScaler(),score_names=ModelStackCV.SCORES_REGRESSION,
#         save_dir='./08-experiments/03-boxscores-players-stats-1st-vs-2nd-half-1997-2023/predict-players-pts-total/',
#         append_to=None
#     )


In [31]:
# PTS_OU_LEVELS = [14.5,19.5,24.5,29.5]
# # Predict by players
# X = {f'pts_ou_{ou}':X_BXSC_PLYR[X_BXSC_PLYR[('H1','pts')]<ou] for ou in PTS_OU_LEVELS} # Select only players who scored less than the ou before the break
# Y = {f'pts_ou_{ou}':(Y_BXSC_PLYR[X_BXSC_PLYR[('H1','pts')]<ou]>ou).astype(int) for ou in PTS_OU_LEVELS} # Convert the ou to a binary classification problem
ou = 5.5
X_BXSC_PLYR[X_BXSC_PLYR[('H1','pts')]<ou]


H1   
                                                                                    mp   
boxscore_id                  team_id              player_id                              
/boxscores/202304090WAS.html /teams/HOU/2023.html /players/s/sengual01.html  13.366667  \
                                                  /players/m/martike04.html  10.866667   
                             /teams/WAS/2023.html /players/t/toddis01.html   23.266667   
                                                  /players/c/cooksxa01.html  18.900000   
                                                  /players/n/nunnke01.html    7.716667   
                                                  /players/g/gillan01.html   10.466667   
/boxscores/202304090TOR.html /teams/MIL/2023.html /players/w/wiggili01.html   9.283333   
                                                  /players/m/matthwe02.html  12.300000   
                             /teams/TOR/2023.html /players/a/achiupr01.html  14.950000   
                                                  /players/k/kolokch01.html   9.050000   
                                                  /players/h/harpero02.html   7.400000   
/boxscores/202304090POR.html /teams/GSW/2023.html /players/l/looneke01.html  13.816667   
                                                  /players/p/paytoga02.html  11.466667   
                             /teams/POR/2023.html /players/m/minayju01.html   7.333333   
                                                  /players/b/butlejo01.html  10.866667   
...                                                                                ...   
/boxscores/199611010DET.html /teams/DET/1997.html /players/l/longgr01.html    9.116667   
                                                  /players/a/augmost01.html  11.966667   
/boxscores/199611010DEN.html /teams/DAL/1997.html /players/k/kiddja01.html   14.700000   
                                                  /players/m/montrer01.html  10.416667   
                                                  /players/m/meyerlo01.html  10.066667   
                                                  /players/m/milleol01.html  13.316667   
                             /teams/DEN/1997.html /players/j/johnser02.html  22.116667   
                                                  /players/j/jacksma01.html  18.550000   
                                                  /players/e/ellisda01.html  20.066667   
/boxscores/199611010BOS.html /teams/CHI/1997.html /players/h/harpero01.html  15.500000   
                                                  /players/k/kukocto01.html  14.050000   
                                                  /players/k/kerrst01.html    8.500000   
                             /teams/BOS/1997.html /players/f/foxri01.html    16.566667   
                                                  /players/e/ellispe01.html  15.050000   
                                                  /players/d/dayto01.html     9.466667   

                                                                                   
                                                                              fg   
boxscore_id                  team_id              player_id                        
/boxscores/202304090WAS.html /teams/HOU/2023.html /players/s/sengual01.html  0.0  \
                                                  /players/m/martike04.html  1.0   
                             /teams/WAS/2023.html /players/t/toddis01.html   1.0   
                                                  /players/c/cooksxa01.html  2.0   
                                                  /players/n/nunnke01.html   2.0   
                                                  /players/g/gillan01.html   2.0   
/boxscores/202304090TOR.html /teams/MIL/2023.html /players/w/wiggili01.html  0.0   
                                                  /players/m/matthwe02.html  0.0   
                             /teams/TOR/2023.html /players/a/achiupr01.html  1.0   
               

In [32]:
(Y_BXSC_PLYR[X_BXSC_PLYR[('H1','pts')]<ou]>ou).astype(int)

boxscore_id                   team_id               player_id                
/boxscores/202304090WAS.html  /teams/HOU/2023.html  /players/s/sengual01.html    1
                                                    /players/m/martike04.html    1
                              /teams/WAS/2023.html  /players/t/toddis01.html     1
                                                    /players/c/cooksxa01.html    1
                                                    /players/n/nunnke01.html     1
                                                    /players/g/gillan01.html     1
/boxscores/202304090TOR.html  /teams/MIL/2023.html  /players/w/wiggili01.html    1
                                                    /players/m/matthwe02.html    0
                              /teams/TOR/2023.html  /players/a/achiupr01.html    1
                                                    /players/k/kolokch01.html    1
                                                    /players/h/harpero02.html    1
/boxscore

In [ ]:
X_BXSC_PLYR.sample(X_BXSC_PLYR.shape[0])